---
# Guía: El juego de las probabilidades

Lea atentamente esta guía y complete los desafíos de código existentes

Nota: este notebook no recomienda ser asíduo a los juegos de apuesta, sólo se cubre este tema por motivos académicos


In [42]:
import pandas as pd # importamos el modulo
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats # importando scipy.stats


In [2]:
np.random.seed(2131986) # para poder replicar el random

%matplotlib inline

### 1. Paradoja del cumpleaños

Un curso de probabilidad y estadística compuesto por 60 alumnos se presenta al examen final. No se sienten muy seguros por lo cual le plantean al profesor la posibilidad de eximirse y aprobar sólo con las notas parciales. A lo cual, el profesor les dice: "OK, hagamos una apuesta. Si en esta sala todos y cada uno de ustedes tiene un cumpleaños un día diferente, entonces los eximo con nota 7. Por otro lado, si más de una persona tiene la misma fecha de cumpleaños, entonces deberán rendir el examen".

Los alumnos hablaron entre ellos y dijeron: "Bueno, son 360 días del año aproximadamente, y somos 60 alumnos, por lo tanto, hay 1/6 de probabilidad de que tengamos la misma fecha de cumpleaños. El profesor tiene las probabilidades en contra. Aceptemos el desafío!"


Calcular la probabilidad de un cumpleaños duplicado puede parecer una tarea desalentadora. Pero ¿qué pasa con calcular la probabilidad de que no haya un cumpleaños duplicado? Esto es realmente una tarea más fácil. Especialmente si simplificamos el problema a un grupo muy pequeño. Supongamos que el grupo sólo tiene una persona, en ese caso, hay una probabilidad del 100% que esta persona no comparte un cumpleaños puesto que no hay nadie más quien compartir. Pero ahora podemos añadir una segunda persona al grupo. ¿Cuáles son las posibilidades de que tenga un cumpleaños diferente de esa persona? De hecho esto es bastante fácil, hay 364 otros días en el año, así que las posibilidades son 364/365. ¿Qué tal si agregamos una tercera persona al grupo? Ahora hay 363/365 días. Para obtener la probabilidad general de que no hay cumpleaños compartidos simplemente multiplicamos las probabilidades individuales (sucesos independientes). Si utilizamos este procedimiento, con la ayuda de Python podemos calcular fácilmente las probabilidades de un cumpleaños compartido en un grupo de 60 personas.

In [19]:
# Con este trozo de codigo podemos chequear la probabilidad
prob = 1.0
asistentes = 2

for i in range(asistentes):
    prob = prob * (365-i)/365 #i=1 364/365

print("Probabilidad de que compartan una misma fecha de cumpleaños es {0:.5f}".format(1 - prob))

Probabilidad de que compartan una misma fecha de cumpleaños es 0.00274


In [ ]:
# Lleve el código anterior a una función que reciba la cantidad de asistentes y retorne la probabilidad de que alguna fecha de cumpleaños coincida
def calcula_proba(asistentes: int) -> float:
    """
    Calcula la probabilidad de que, en un grupo de 'asistentes', al menos dos compartan fecha de cumpleaños.
    Asume 365 días equiprobables y eventos independientes.

    Retorna un float entre 0 y 1.
    """
    if asistentes <= 1:
        return 0.0
    if asistentes >= 365:
        # P(duplique) ~ 1 si asistentes >= 365 por principio del palomar (pigeonhole)
        return 1.0
    prob_no_coinciden = 1.0
    for i in range(asistentes):
        prob_no_coinciden *= (365 - i) / 365
    return 1 - prob_no_coinciden

.# ejemplo rápido
print(f"Probabilidad con 2 asistentes: {calcula_proba(2):.5f}")

Complete la siguiente tabla con las probabilidades calculadas de acuerdo a la cantidad de asistentes

| N° Asistentes | Probabilidad |
| ------------- | ------------ |
| 1             | 0.0000       |
| 2             |        |
| 3             |        |
| 4             |        |
| 5             |        |
| 6             |        |
| 7             |        |
| 8             |        |
| 9             |        |
| 10             |        |




In [ ]:
## haga un código que itere desde 1 hasta 100 participantes y realice un diagraga donde aparezca la probabilidad para cada numero de asistentes iterado
nums = list(range(1, 101))
probs = [calcula_proba(n) for n in nums]
plt.figure(figsize=(8,4))
plt.plot(nums, probs, marker='o', markersize=3)
plt.axhline(0.5, color='red', linestyle='--', label='0.5')
plt.title('Probabilidad de cumpleaños compartido vs asistentes')
plt.xlabel('Número de asistentes')
plt.ylabel('Probabilidad')
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

In [ ]:
## ¿Cuántos participantes se requieren para tener una probabilidad de al menos el 50% de que hayan dos personas que compartan la fecha de cumpleaños?
target = 0.5
n = 1
while calcula_proba(n) < target:
    n += 1
print(f"Se requieren {n} participantes para alcanzar >= 50% (prob = {calcula_proba(n):.4f}).")

In [ ]:
## ¿Qué probabilidad tiene el curso de eximirse del examen?
## Eximirse si todos tienen cumpleaños distinto -> prob_no_coinciden; aquí se pide el contrario según enunciado; mostramos ambos.
asistentes = 60
prob_no_coinciden = 1.0
for i in range(asistentes):
    prob_no_coinciden *= (365 - i) / 365
prob_duplique = 1 - prob_no_coinciden
print(f"Con {asistentes} alumnos:")
print(f" - Probabilidad de NO compartir cumpleaños (eximirse): {prob_no_coinciden:.6f}")
print(f" - Probabilidad de compartir al menos un cumpleaños: {prob_duplique:.6f}")

La paradoja del cumpleaños es un problema muy conocido en el campo de la teoría de probabilidad. Plantea las siguientes interesantes preguntas: ¿Cuál es la probabilidad de que, en un grupo de personas elegidas al azar, al menos dos de ellas habrán nacido el mismo día del año? ¿Cuántas personas son necesarias para asegurar una probabilidad mayor al 50%?. Excluyendo el 29 de febrero de nuestros cálculos y asumiendo que los restantes 365 días de posibles cumpleaños son igualmente probables, nos sorprendería darnos cuenta de que, en un grupo de sólo 23 personas, la probabilidad de que dos personas compartan la misma fecha de cumpleaños es mayor al 50%!!!.

### 2. Let's Make a Deal!!

El problema de Monty Hall es un famoso problema de probabilidad que lleva el nombre del presentador de televisión estadounidense Monty Hall, quien era conocido por su participación en el programa de juegos "Let's Make a Deal" en la década de 1960.

El problema se plantea de la siguiente manera:

Imagina que estás en un programa de juegos donde tienes que elegir una de tres puertas. Detrás de una de las puertas hay un premio grande, como un automóvil, mientras que detrás de las otras dos puertas hay premios menos valiosos, como cabras. El presentador, que sabe lo que hay detrás de cada puerta, te permite elegir una puerta, pero antes de abrir la puerta que elegiste, abre una de las otras dos puertas que sabes que no tiene el premio grande y revela que hay una cabra detrás de esa puerta.

Ahora, el presentador te da la opción de quedarte con la puerta que originalmente elegiste o cambiar a la otra puerta que aún no ha sido abierta. ¿Qué deberías hacer? ¿Deberías quedarte con tu elección original o cambiar a la otra puerta?

El problema de Monty Hall genera mucha controversia y confusión, ya que parece que cambiar de puerta no debería afectar tus probabilidades de ganar. Sin embargo, de acuerdo con el razonamiento probabilístico, cambiar de puerta en realidad te da una mayor probabilidad de ganar el premio grande. La solución correcta al problema de Monty Hall implica que siempre es mejor cambiar de puerta después de que el presentador revele una de las otras puertas.

Este problema se ha convertido en un clásico en la teoría de la probabilidad y ha generado mucho interés y debate entre matemáticos, estadísticos y entusiastas de los juegos.

In [13]:
# clase que implementa el juego de Monty Hall
def elegir_puerta():
    """
    Función para elegir una puerta. Devuelve 1, 2, o 3 en forma aleatoria.
    """
    return np.random.randint(1,4)

class MontyHall:
    """
    Clase para modelar el problema de Monty Hall.
    """
    def __init__(self):
        """
        Crea la instancia del problema. 
        """
        # Elige una puerta en forma aleatoria.
        self.puerta_ganadora = elegir_puerta()
        # variables para la puerta elegida y la puerta descartada
        self.puerta_elegida = None
        self.puerta_descartada = None
 
    def selecciona_puerta(self):
        """
        Selecciona la puerta del concursante en forma aleatoria.
        """
        self.puerta_elegida = elegir_puerta()
 
    def descarta_puerta(self):
        """
        Con este método el presentador descarta una de la puertas.
        """
        # elegir puerta en forma aleatoria .
        d = elegir_puerta()
        # Si es al puerta ganadora o la del concursante, volver a elegir.
        while d == self.puerta_ganadora or d == self.puerta_elegida:
            d = elegir_puerta()
        # Asignar el valor a puerta_descartada.
        self.puerta_descartada = d
 
    def cambiar_puerta(self):
        """
        Cambia la puerta del concursante una vez que se elimino una puerta.
        """
        # 1+2+3=6. Solo existe una puerta para elegir.
        self.puerta_elegida = 6 - self.puerta_elegida - self.puerta_descartada
 
    def gana_concursante(self):
        """
        Determina si el concursante gana. 
        Devuelve True si gana, False si pierde.
        """
        return self.puerta_elegida == self.puerta_ganadora
            
 
    def jugar(self, cambiar=True):
        """
        Una vez que la clase se inicio, jugar el concurso.
 
        'cambiar' determina si el concursante cambia su elección.
        """
        # El concursante elige una puerta.
        self.selecciona_puerta()
        # El presentador elimina una puerta.
        self.descarta_puerta()
        # El concursante cambia su elección.
        if cambiar:
            self.cambiar_puerta()
        # Determinar si el concursante ha ganado.
        return self.gana_concursante()

In [ ]:
# utilice la clase anterior para simular algunas partidas de juego. ¿Qué resultados se obtienen?





Este problema, también es un problema muy popular dentro de la teoría de probabilidad; y se destaca por su solución que a simple vista parece totalmente anti-intuitiva. Intuitivamente, es bastante sencillo que nuestra elección original (cuando hay tres puertas para elegir) tiene una probabilidad de 1/3 de ganar el concurso. Las cosas sin embargo se complican, cuando se descarta una puerta. Muchos dirían que ahora tenemos una probabilidad de 1/2 de ganar, seleccionando cualquiera de las dos puertas; pero este no es el caso. Un aspecto crítico del problema es darse cuenta de que la elección de la puerta a descartar por el presentador, no es una decisión al azar. El presentador puede descartar una puerta porque él sabe (a) qué puerta hemos seleccionado y (b) qué puerta tiene la ferrari. De hecho, en muchos casos, el presentador debe quitar una puerta específica. Por ejemplo, si seleccionamos la puerta 1 y el premio está detrás de la puerta 3, el presentador no tiene otra opción que retirar la puerta 2. Es decir, que la elección de la puerta a descartar está condicionada tanto por la puerta con el premio como por la puerta que seleccionamos inicialmente. Este hecho, cambia totalmente la naturaleza del juego, y hace que las probabilidades de ganar sean 2/3 si cambiamos de puerta!.

In [ ]:
# Ahora, jugamos el concurso. primero nos vamos a quedar con nuestra elección
# inicial. Vamos a ejecutar el experimiento 10.000 veces.

gana, pierde = 0, 0

n_experimentos = 10000
for _ in range(n_experimentos):
    juego = MontyHall()
    resultado = juego.jugar(cambiar=False)
    if resultado:
        gana += 1
    else:
        pierde += 1

# veamos la fecuencia de victorias del concursante.
porc_gana = 100.0 * gana / (gana + pierde)
 
print("\n10.000 concursos sin cambiar de puerta:")
print("  gana: {0:} concursos".format(gana))
print("  pierde: {0:} concursos".format(pierde))
print("  probabilidad: {0:.2f} porcentaje de victorias".format(porc_gana))


In [ ]:
# Ahora, jugamos el concurso siempre cambiando la elección inicial
# Vamos a ejecutar el experimiento 10.000 veces.
gana, pierde = 0, 0

n_experimentos = 10000
for _ in range(n_experimentos):
    juego = MontyHall()
    resultado = juego.jugar(cambiar=True)
    if resultado:
        gana += 1
    else:
        pierde += 1

# veamos la fecuencia de victorias del concursante.
porc_gana = 100.0 * gana / (gana + pierde)
 
print("\n10.000 concursos cambiando de puerta:")
print("  gana: {0:} concursos".format(gana))
print("  pierde: {0:} concursos".format(pierde))
print("  probabilidad: {0:.2f} porcentaje de victorias".format(porc_gana))

Como esta simulación lo demuestra, si utilizamos la estrategia de siempre cambiar de puerta, podemos ganar el concurso un 66% de las veces; mientras que si nos quedamos con nuestra elección inicial, solo ganaríamos el 33% de las veces.

In [23]:
# Comente sus conclusiones del experimento. ¿Está de acuerdo con lo recién planteado?





### 3. Juego del Color

En la ruleta, existe la posibilidad de apostar al color (rojo o negro). Supongamos que la probablidad es de 50% (no existe el cero verde) para simplificar este análisis. Un apostador frecuente a apostar al color, posiblemente esté pensando que si un color ya salió en el juego previo, es más probable que aparezca el otro color. En este experimento vamos a develar si esta estrategia permite obtener resultados o bien es una falacea.

Para esto, supongamos que realizamos dos lanzamientos de la ruleta. Encuentre:

- La probabilidad de que ambos colores sean rojo, dado que el primer lanzamiento fue rojo.
- La probabilidad de que ambos colores sean rojo, dado que al menos uno de los lanzamientos fue rojo


In [ ]:
import numpy as np

#Se generan varias repeticiones del experimento
n = int(1e6)
resultados = np.random.choice(['rojo','negro'], size = (n,2))
resultados


In [ ]:
#Se obtienen los experimentos en los cuales la primera es rojo
condicional = resultados[resultados[:,0] == 'rojo']
condicional


In [ ]:
#Se obtienen los experimentos en donde los dos colores son rojo
ambas_cara = resultados[np.logical_and(resultados[:,0] == 'rojo',
                                      resultados[:,1] == 'rojo')]
ambas_cara

In [ ]:
# A) Calcular la probabilidad de que ambos colores sean rojo, dado que el primer lanzamiento fue rojo
# P(cara2/cara1) = P(cara1 interseccion cara2)/p(cara1)
proba = len(ambas_cara) / len(condicional)
print('La probabilidad condicional es', round(proba,2))



In [ ]:
# B) La probabilidad de que ambos colores sean rojo, dado que al menos uno de los lanzamientos fue rojo
#Se obtienen los experimentos en los cuales al menos uno es rojo
aux = np.logical_or(resultados[:,0] == 'rojo', resultados[:,1] == 'rojo')
aux

In [ ]:
condicional = resultados[aux]
condicional

In [ ]:
#Calcula la probabilidad condicional
condicional = resultados[aux]
ambas_rojo = resultados[np.logical_and(resultados[:,0] == 'rojo', resultados[:,1] == 'rojo')]
proba = len(ambas_rojo) / len(condicional)
print('La probabilidad condicional es', round(proba,2))

In [32]:
# ¿Qué se puede concluir de la estrategia apostadora planteada?







In [ ]:
# ¿Qué pasaría si nos fijáramos en los casos en que han salido 3 colores negro consecutivos? ¿Tiene más probabilidad de salir rojo en el siguiente lanzamiento?
# Fundamente su respuesta







### 4. Simulación de la Martingala

La martingala, es una técnica de apuesta en que si se pierde una apuesta, se aumenta la apuesta a la siguiente para recuperar el valor perdido, suponiendo que no se pueden tener demasiadas pérdidas consecutivas y en algún momento se ganará la apuesta, lo que permitirá recuperar las pérdidas. Esto podría funcionar en apuestas que tienen 50% de probabilidad de ganar, como por ejemplo, lanzar una moneda, apostar a los colores en la ruleta, o bien a los pares o impares.

Las reglas a simular son las siguientes:
- Suponga que usted tiene un total de 100 fichas para jugar
- Las apuestas las realiza de 1 ficha
- Sin embargo, si usted pierde una apuesta, al lanzamiento siguiente apuesta lo que perdió en el lanzamiento anterior más una ficha
- La simulación termina con 10.000 lanzamientos o bien con la pérdida de todas las fichas

Se pide:
- Hagra un gráfico de la evolución de la cantidad de fichas en el tiempo
- Grafique los resultados
- ¿Cómo evalúa la técnica de la Martingala? 


In [ ]:
# Simulación de la Martingala
import random
fichas_iniciales = 100
max_lanzamientos = 10_000
fichas = fichas_iniciales
apuesta = 1
historial = [fichas]
for _ in range(max_lanzamientos):
    # Resultado del juego (50% ganar/perder)
    gana = random.random() < 0.5
    if gana:
        fichas += apuesta
        apuesta = 1  # resetear apuesta tras ganar
    else:
        fichas -= apuesta
        apuesta = min(fichas if fichas>0 else 0, apuesta + 1)  # aumentar apuesta si pierde, limitado por fichas
    historial.append(fichas)
    if fichas <= 0:
        break
len(historial)

In [ ]:
plt.figure(figsize=(9,4))
plt.plot(historial, color='steelblue')
plt.title('Evolución de fichas con Martingala')
plt.xlabel('Lanzamiento')
plt.ylabel('Fichas')
plt.grid(True, alpha=0.3)
plt.axhline(fichas_iniciales, color='gray', linestyle='--', label='Fichas iniciales')
plt.legend()
plt.show()

---